# Amazon Bedrock boto3 Prerequisites

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

---

In this demo notebook, we demonstrate how to use the [`boto3` Python SDK](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html) to work with [Amazon Bedrock](https://aws.amazon.com/bedrock/) Foundation Models.

---


# Amazon Bedrock boto3 전제 조건

>  이 노트북은 SageMaker Studio*의 *`Data Science 3.0`* * 커널 *과 잘 작동합니다* .

 이 데모 노트북에서는 [`boto3` Python SDK를](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html) 사용하여 [Amazon Bedrock](https://aws.amazon.com/bedrock/) Foundation Models로 작업하는 방법을 보여줍니다.



## 전제조건

 이 섹션의 셀을 실행하여 이 워크숍의 노트북에 필요한 패키지를 설치하세요. ⚠️ pip 종속성 오류가 표시됩니다. 이러한 오류는 무시해도 됩니다. ⚠️

 무시 오류: pip의 종속성 해결 프로그램은 현재 설치된 모든 패키지를 고려하지 않습니다. 이 동작은 다음 종속성 충돌의 원인입니다.


In [ ]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"



## boto3 클라이언트 만들기

 Bedrock API와의 상호 작용은 Python용 AWS SDK: [boto3 를](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html) 통해 수행됩니다.

####  다른 클라이언트 사용

 boto3은 Amazon Bedrock이 다양한 작업을 수행할 수 있도록 다양한 클라이언트를 제공합니다. [`InvokeModel`](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_InvokeModel.html) 및 [`InvokeModelWithResponseStream`](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_InvokeModelWithResponseStream.html) 대한 작업은 Amazon Bedrock Runtime에서 지원되며 <br>
 [ListFoundationModels](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_ListFoundationModels.html) 와 같은 다른 작업은 [Amazon Bedrock 클라이언트를](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_Operations_Amazon_Bedrock.html) 통해 처리됩니다.
 

####  기본 자격 증명 체인 사용

 [Amazon Sagemaker Studio](https://aws.amazon.com/sagemaker/studio/) 에서 이 노트북을 실행 중이고 Sagemaker Studio [실행 역할에](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) Bedrock에 액세스할 수 있는 권한이 있는 경우 아래 셀을 있는 그대로 실행할 수 있습니다. 이는 기본 AWS 자격 증명이 Bedrock에 액세스할 수 있는 컴퓨터에서 이러한 노트북을 실행하는 경우에도 마찬가지입니다.


In [ ]:
import json
import os
import sys

import boto3

boto3_bedrock = boto3.client('bedrock')


#### 연결 확인

 우리가 사용할 수 있는 모든 모델을 알려주는 `list_foundation_models()` 메서드를 사용해 클라이언트가 작동하는지 확인할 수 있습니다.


In [ ]:
boto3_bedrock.list_foundation_models()



## `InvokeModel` 본문 및 출력

 Amazon Bedrock 런타임 클라이언트( `InvokeModel` API)의 `invoke_model()` 메서드는 우리가 사용하는 모델에 관계없이 대부분의 텍스트 생성 및 처리 작업에 사용하는 기본 메서드가 될 것입니다.

 방법은 공유되지만 입력 및 출력 형식은 사용된 기반 모델에 따라 달라집니다. 아래에 설명되어 있습니다.


### Amazon Titan Large

#### Input
```json
{   
    "inputText": "<prompt>",
    "textGenerationConfig" : { 
        "maxTokenCount": 512,
        "stopSequences": [],
        "temperature": 0.1,  
        "topP": 0.9
    }
}
```

#### Output

```json
{
    "inputTextTokenCount": 613,
    "results": [{
        "tokenCount": 219,
        "outputText": "<output>"
    }]
}
```

### Anthropic Claude

#### Input

```json
{
    "prompt": "\n\nHuman:<prompt>\n\nAnswer:",
    "max_tokens_to_sample": 300,
    "temperature": 0.5,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman:"]
}
```

#### Output

```json
{
    "completion": "<output>",
    "stop_reason": "stop_sequence"
}
```

### Stability AI Stable Diffusion XL

#### Input

```json
{
    "text_prompts": [
        {"text": "this is where you place your input text"}
    ],
    "cfg_scale": 10,
    "seed": 0,
    "steps": 50
}
```

#### Output

```json
{ 
    "result": "success", 
    "artifacts": [
        {
            "seed": 123, 
            "base64": "<image in base64>",
            "finishReason": "SUCCESS"
        },
        //...
    ]
}
```


## 일반적인 추론 매개변수 정의

###  무작위성과 다양성

 FM은 일반적으로 다음 매개변수를 지원하여 응답의 무작위성과 다양성을 제어합니다.


 **Temperature** – 대규모 언어 모델은 확률을 사용하여 단어를 순서대로 구성합니다. 주어진 다음 단어에 대해 시퀀스의 다음 단어에 대한 옵션의 확률 분포가 있습니다. 온도를 0에 가깝게 설정하면 모델은 확률이 더 높은 단어를 선택하는 경향이 있습니다. 온도를 0에서 더 멀리 설정하면 모델이 확률이 낮은 단어를 선택할 수 있습니다. 기술적인 측면에서 온도는 온도 샘플링 기술을 구현하여 다음 토큰에 대한 확률 밀도 함수를 변조합니다. 이 매개변수는 밀도 함수 곡선을 깊게 하거나 평평하게 만들 수 있습니다. 값이 낮을수록 더 결정적인 반응을 갖는 가파른 곡선이 되고, 값이 높을수록 무작위 반응이 더 많은 평평한 곡선이 됩니다.

 **Top K** - 온도는 잠재적인 단어의 확률 분포를 정의하고 상위 K는 모델이 더 이상 단어를 선택하지 않는 지점을 정의합니다. 예를 들어, K=50이면 모델은 주어진 순서에서 다음에 올 가능성이 가장 높은 단어 50개 중에서 선택합니다. 이렇게 하면 특이한 단어가 순서대로 다음에 선택될 확률이 줄어듭니다. 기술적인 측면에서 Top K는 Top-K 필터링을 위해 보관할 확률이 가장 높은 어휘 토큰의 수입니다. 이는 가능한 토큰의 배포를 제한하므로 모델은 확률이 가장 높은 토큰 중 하나를 선택합니다.


 **Top P** – Top P는 잠재적인 선택의 확률 합계를 기반으로 컷오프를 정의합니다. 상위 P를 1.0 미만으로 설정하면 모델은 가능성이 가장 높은 옵션을 고려하고 가능성이 낮은 옵션을 무시합니다. Top P는 Top K와 유사하지만 선택 항목 수를 제한하는 대신 확률의 합을 기준으로 선택 항목을 제한합니다. 예를 들어 "발굽 소리가 들립니다."라는 프롬프트의 경우 모델이 다음 단어로 "말", "얼룩말" 또는 "유니콘"을 제공하도록 할 수 있습니다. Top K 또는 Top P를 제한하지 않고 온도를 최대로 설정하면 "유니콘"과 같은 특이한 결과를 얻을 확률이 높아집니다. 온도를 0으로 설정하면 "말"이 나올 확률이 높아집니다. 온도를 높게 설정하고 Top K나 Top P를 최대로 설정하면 '말'이나 '얼룩말'이 나올 확률은 높아지고, '유니콘'이 나올 확률은 낮아집니다.


###  길이

 다음 매개변수는 생성된 응답의 길이를 제어합니다.

 **Response length** – 생성된 응답에 사용할 최소 및 최대 토큰 수를 구성합니다.


 **Length penalty** – 길이 페널티는 더 긴 응답에 페널티를 적용하여 출력이 더욱 간결해지도록 모델을 최적화합니다. 

 기술적인 측면에서 length panalty 는 긴 응답에 대해 기하급수적으로 모델에 페널티를 줍니다. 0.0은 페널티가 없음을 의미합니다. 모델이 더 긴 시퀀스를 생성하려면 0.0보다 작은 값을 설정하고, 더 짧은 시퀀스를 생성하려면 모델이 0.0보다 큰 값을 설정하십시오.

###  반복

 다음 매개변수는 생성된 응답의 반복을 제어하는 데 도움이 됩니다.

 **Repetition penalty (presence penalty)** - 응답에서 동일한 단어(토큰)의 반복을 방지합니다. 
 1.0은 페널티가 없음을 의미합니다. 1.0보다 크면 반복이 감소합니다.



## 모델을 사용해 보세요

 몇 가지 이론을 배제하고 실제 모델을 살펴보겠습니다! 아래 셀을 실행하여 각 모델에 대한 기본 동기식 호출 예시를 확인하세요.


In [ ]:
import boto3
import botocore
import json 

bedrock_runtime = boto3.client('bedrock-runtime')


### Anthropic Claude 모델을 이용한 추론 

In [ ]:
# If you'd like to try your own prompt, edit this parameter!
prompt_data = """Human: 리더로서 강력한 비즈니스 결정을 내리는 방법에 대한 블로그를 작성해 주세요.

Assistant:
"""


In [ ]:
body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 500})
modelId = "anthropic.claude-instant-v1"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

try:

    response = bedrock_runtime.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())

    print(response_body.get("completion"))

except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error


### Stability Stable Diffusion XL 을 이용한 추론

In [ ]:
prompt_data = "snowman in mountain"
body = json.dumps({
    "text_prompts": [{"text": prompt_data}],
    "cfg_scale": 10,
    "seed": 20,
    "steps": 50
})
modelId = "stability.stable-diffusion-xl-v1"
accept = "application/json"
contentType = "application/json"

try:

    response = bedrock_runtime.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())

    print(response_body["result"])
    print(f'{response_body.get("artifacts")[0].get("base64")[0:80]}...')

except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error



**참고:** 출력은 이미지 데이터의 [base64 인코딩](https://docs.python.org/3/library/base64.html) 문자열입니다. 아래 예와 같이 임의의 이미지 처리 라이브러리(예: [Pillow](https://pillow.readthedocs.io/en/stable/) )를 사용하여 이미지를 디코딩할 수 있습니다.


In [ ]:
import base64
import io
from PIL import Image

base_64_img_str = response_body.get("artifacts")[0].get("base64")
image = Image.open(io.BytesIO(base64.decodebytes(bytes(base_64_img_str, "utf-8"))))
image

## Generate streaming output

For large language models, it can take noticeable time to generate long output sequences. Rather than waiting for the entire response to be available, latency-sensitive applications may like to **stream** the response to users.

Run the code below to see how you can achieve this with Bedrock's `invoke_model_with_response_stream()` method - returning the response body in separate chunks.


## 스트리밍 출력 생성

 대규모 언어 모델의 경우 긴 출력 시퀀스를 생성하는 데 상당한 시간이 걸릴 수 있습니다. 대기 시간에 민감한 애플리케이션은 전체 응답이 제공될 때까지 기다리는 대신 사용자에게 응답을 **스트리밍** 하는 것을 선호할 수 있습니다.

 아래 코드를 실행하여 Bedrock의 `invoke_model_with_response_stream()` 메서드(응답 본문을 별도의 청크로 반환)를 사용하여 이를 달성하는 방법을 확인하세요.


In [ ]:
from IPython.display import clear_output, display, display_markdown, Markdown
prompt_data = """Command: 리더로서 강력한 비즈니스 결정을 내리는 방법에 대한 블로그를 작성해 주세요.

Blog:
"""

body = json.dumps({"inputText": prompt_data})
#modelId = "amazon.titan-tg1-large"  # (Change this, and the request body, to try different models)
modelId = "anthropic.claude-instant-v1"  # (Change this, and the request body, to try different models)
accept = "application/json"
contentType = "application/json"

try:

    response = bedrock_runtime.invoke_model_with_response_stream(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    stream = response.get('body')
    output = []

    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                chunk_obj = json.loads(chunk.get('bytes').decode())
                if 'outputText' in chunk_obj:
                    text = chunk_obj.get('outputText', None)
                    print(text,end='')
                    if not text :
                        break
                    #text = chunk_obj['outputText']
                    clear_output(wait=True)
                    output.append(text)
                    display_markdown(Markdown(''.join(output)))

except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error


### Anthropic Claude (messages API)

In [ ]:
# If you'd like to try your own prompt, edit this parameter!
prompt_data = """Human: 리더로서 강력한 비즈니스 결정을 내리는 방법에 대해 500자 문단으로 작성해 주세요..

Assistant:
"""


In [ ]:
messages_API_body = {
    "anthropic_version": "bedrock-2023-05-31", 
    "max_tokens": 512,
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt_data
                }
            ]
        }
    ]
}

In [ ]:
from IPython.display import clear_output, display, display_markdown, Markdown

body = json.dumps(messages_API_body)
modelId = "anthropic.claude-v2"  # (Change this to try different model versions)
accept = "application/json"
contentType = "application/json"

try:

    response = bedrock_runtime.invoke_model_with_response_stream(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    
    stream = response.get('body')
    
    
    output = []

    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                chunk_obj = json.loads(chunk.get('bytes').decode())
                if 'delta' in chunk_obj:
                    delta_obj = chunk_obj.get('delta', None)
                    if delta_obj:
                        text = delta_obj.get('text', None)
                        print(text,end='')
                        if not text :
                            break
                    # output.append(text[0]) if type(text) is list and len(text)>0 else output.append('')
                    # display_markdown(Markdown(text))

except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error


## Generate embeddings


텍스트 임베딩을 사용하여 텍스트를 의미 있는 벡터 표현으로 변환합니다. 텍스트 본문을 입력했습니다.
출력은 (1 x n) 벡터입니다. 다양한 애플리케이션에 임베딩 벡터를 사용할 수 있습니다.
Bedrock은 현재 텍스트 유사성을 지원하는 텍스트 포함을 위한 Titan Embedding을 제공합니다(텍스트 본문 간의 의미적 유사성)과 텍스트 검색(예: 검색).

이 글을 쓰는 시점에는 API를 통해 `amazon.titan-embed-text-v1` 을 임베딩 모델로 사용할 수 있습니다. 
입력 텍스트 크기는 8192 토큰이고 출력 벡터 길이는 1536입니다.

텍스트 임베딩 모델을 사용하려면 InvokeModel API 작업 또는 Python SDK를 사용하십시오.
InvokeModel을 사용하여 지정된 모델에서 입력 텍스트의 벡터 표현을 검색합니다.


#### Input

```json
{
    "inputText": "<text>"
}
```

#### Output

```json
{
    "embedding": []
}
```



일부 텍스트의 임베딩을 생성하는 방법을 살펴보겠습니다.


In [ ]:
prompt_data = "Amazon Bedrock supports foundation models from industry-leading providers such as \
AI21 Labs, Anthropic, Stability AI, and Amazon. Choose the model that is best suited to achieving \
your unique goals."


In [ ]:
body = json.dumps({"inputText": prompt_data})
modelId = "amazon.titan-embed-text-v1"  # (Change this to try different embedding models)
accept = "application/json"
contentType = "application/json"

try:

    response = bedrock_runtime.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())

    embedding = response_body.get("embedding")
    print(f"The embedding vector has {len(embedding)} values\n{embedding[0:3]+['...']+embedding[-3:]}")

except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error



## 다음 단계

 이 노트북에서는 AWS Python SDK를 사용하여 Amazon Bedrock 모델을 호출하는 몇 가지 기본 예를 보여주었습니다. 이제 다른 랩을 탐색하여 다양한 사용 사례와 패턴을 자세히 알아볼 준비가 되었습니다.
